<a href="https://colab.research.google.com/github/Dowell-Lab/psea/blob/main/notebook_example/one_comorbid_many_gene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages we will use

In [48]:
import pandas as pd
import plotly.express as px
from scipy.stats import zscore


# read in the files we will look at

In [2]:
url="https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/psea_scores_20240923-152820.adjpval.csv"
df = pd.read_csv(url, index_col=0)
url2="https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/genes.csv"
genedf = pd.read_csv(url2, index_col=0)

In [3]:
genedf["value"]=genedf["gene_id"]

In [4]:
#this brings in the medical disorders for all the individuals with Trisomy 21 in the Human Trisome Project
url = "https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/comorbid_file.csv"
comorbid_df = pd.read_csv(url, index_col=0)



In [5]:
#This brings in the normalized counts for all the individuals with Trisomy 21 in the Human Trisome Project. These are not the real count data but are similar to reall count data.
#df=pd.read_csv('/content/drive/MyDrive/normcounts.csv')
url_counts="https://media.githubusercontent.com/media/Dowell-Lab/psea/refs/heads/main/testdata/value_expression_large.csv"
gene_exp_df=pd.read_csv(url_counts, index_col=0)



# Create a data frame of each gene linked to a comorbid

In [98]:
def acomorbidanditsgenes(acomorbid, gene_exp_df, comorbid_df, df, genedf, cuttoff=0.1, adjpvalcol="p_value_BenjaminiHochberg"):
  df_nona = df.dropna()
  df_nona_threshold = df_nona[df_nona[adjpvalcol]<cuttoff]
  df_nona_threshold_names = df_nona_threshold.merge(genedf, how ="inner", on="value")
  aconditiondf = df_nona_threshold_names[df_nona_threshold_names["binary_attribute"]==acondition]
  aconditiondf = aconditiondf.sort_values(["NES"])
  geneensmbllist = sorted(aconditiondf["value"].to_list())
  patientgenesexpdf = gene_exp_df[geneensmbllist+["Patient"]]
  onecomborbiddf = comorbid_df[[acomorbid, "Patient"]]
  merge_df = onecomborbiddf.merge(patientgenesexpdf, on="Patient")
  return merge_df


def splitdf_high_low_genes(thisdf, acondition, df,cuttoff=0.1, adjpvalcol="p_value_BenjaminiHochberg"):
  df_nona = df.dropna()
  df_nona_threshold = df_nona[df_nona[adjpvalcol]<cuttoff]
  df_nona_threshold_names = df_nona_threshold.merge(genedf, how ="inner", on="value")
  aconditiondf = df_nona_threshold_names[df_nona_threshold_names["binary_attribute"]==acondition]
  aconditiondf = aconditiondf.sort_values(["NES"])
  highgenes = aconditiondf[aconditiondf["NES"]<0]["value"].to_list()
  lowgenes = aconditiondf[aconditiondf["NES"]>0]["value"].to_list()
  high_merge_df = thisdf[[acondition, "Patient"]+highgenes]
  low_merge_df = thisdf[[acondition, "Patient"]+lowgenes]
  return high_merge_df, low_merge_df

def splitpeoplewithandwithout(acomorbid, adfofexpressionandcomorbid):
  withcomorid = adfofexpressionandcomorbid[adfofexpressionandcomorbid[acomorbid]==1].copy()
  without_comorid = adfofexpressionandcomorbid[adfofexpressionandcomorbid[acomorbid]==0].copy()
  withcomorid = withcomorid.drop(columns=["Patient", acomorbid])
  without_comorid = without_comorid.drop(columns=["Patient", acomorbid])
  return withcomorid, without_comorid

def zscoregenes(acomorbid, genexpconditiondf):
    dontuse_columns = ["Patient", acomorbid]
    gene_names = [thiscolname for thiscolname in genexpconditiondf.columns if thiscolname not in dontuse_columns]
    #df.apply(zscore)
    transpose_df = genexpconditiondf.drop(columns=dontuse_columns)
    transpose_df = transpose_df.apply(zscore)
    transpose_df["Patient"] = genexpconditiondf["Patient"]
    transpose_df[acomorbid] = genexpconditiondf[acomorbid]
    transpose_df = transpose_df[["Patient", acomorbid]+gene_names]
    return transpose_df


# Pick a binary_attribute to look at

In [141]:
#acondition = "constipation_disorder"
#acondition = "strabismus"
acondition="skeletal_system_disorder"
genexpconditiondf = acomorbidanditsgenes(acondition,gene_exp_df, comorbid_df, df, genedf)
zscore_genexpconditiondf = zscoregenes(acondition, genexpconditiondf)
zscore_genexpconditiondf_high, zscore_genexpconditiondf_low = splitdf_high_low_genes(zscore_genexpconditiondf, acondition, df)
lookatdf = zscore_genexpconditiondf_high



In [142]:
from sklearn import cluster
data = lookatdf.drop(columns = ["Patient", acondition])
k_means = cluster.KMeans(n_clusters=5, max_iter=50, random_state=1)
k_means.fit(data)
labels = k_means.labels_
whichcluster = pd.DataFrame(labels, index=lookatdf.Patient, columns=['Cluster_ID'])
whichcluster.reset_index(inplace=True)
lookatdf = lookatdf.merge(whichcluster, on="Patient")
lookatdf = lookatdf.sort_values(["Cluster_ID"])
lookatdf = lookatdf.drop(columns=["Cluster_ID"])
wco, woco = splitpeoplewithandwithout(acondition, lookatdf)


In [143]:
wco.reset_index(inplace=True, drop=True)

In [144]:
wco

,ENSG00000159086,ENSG00000185658,ENSG00000159140,ENSG00000234883,ENSG00000182670,ENSG00000236532,ENSG00000159267,ENSG00000173276,ENSG00000160299,ENSG00000159131,...,ENSG00000156239,ENSG00000234509,ENSG00000154727,ENSG00000180530,ENSG00000270116,ENSG00000237945,ENSG00000205930,ENSG00000159216,ENSG00000197381,ENSG00000223078
0,1.580344,1.879416,0.752485,2.580518,0.483948,0.687941,0.911940,0.890386,1.505634,0.324415,...,1.813940,0.691696,1.851384,1.656984,0.172798,0.761147,0.761815,-1.121225,1.176070,2.214724
1,0.953409,1.631708,1.304788,2.504443,1.133702,-0.476018,3.930850,0.337962,2.497979,1.728810,...,-0.481859,-0.166078,1.139396,-0.075114,0.591683,1.360089,-0.910100,2.331179,-0.169250,0.115252
2,0.858061,1.131403,0.791720,-0.196263,1.580583,2.477043,2.606617,2.005083,0.352954,1.849523,...,0.681479,0.470481,0.505028,1.554584,0.511206,0.099547,1.214104,2.315758,2.017117,1.456053
3,1.656679,1.401054,2.010008,1.598771,0.578618,0.773528,0.047928,1.441073,2.195144,-0.104943,...,0.909091,0.511042,1.716294,0.367489,-0.365306,0.377645,-0.012223,-0.341792,0.234611,0.976640
4,1.635587,1.286275,1.992055,0.306582,1.870871,-0.104046,0.049344,2.214577,0.257386,0.240016,...,1.048833,1.011977,1.223112,1.524489,0.617020,-0.013409,1.060909,1.116026,1.171613,-0.864674
5,1.646740,1.931174,0.846566,0.946844,1.172465,0.579940,1.265753,1.627807,-0.098699,2.099249,...,3.342570,2.066504,1.637964,2.178165,0.475840,0.030623,1.134079,-0.923595,0.860417,-0.538521
6,1.776865,1.438709,2.066763,1.215698,2.345144,1.262244,1.908862,0.332041,0.673186,0.791794,...,2.119890,1.109423,1.960936,1.607161,2.996090,0.969861,0.638432,0.385691,2.429554,1.705030
7,1.055536,2.137033,1.209133,0.912668,1.534625,0.094300,0.699122,1.465768,1.787642,1.424015,...,1.118212,0.297788,1.507368,1.107497,1.391997,0.736168,1.424795,1.208007,1.067386,2.557607
8,1.594634,0.945900,1.250807,1.239681,0.733058,0.288295,0.673356,1.104999,2.091502,0.449532,...,0.546581,0.575843,1.305629,-0.125846,-0.076362,1.545866,-0.216245,-0.321349,0.068321,2.114544
9,1.204042,1.922960,0.206336,1.385333,1.802976,0.383209,1.444324,0.830008,1.127698,1.536034,...,1.418991,1.131032,1.299830,1.121886,0.364703,1.812765,0.871131,0.467129,1.717685,0.258425


In [145]:
woco.reset_index(inplace=True, drop=True)

In [146]:

fig = px.imshow(wco)
fig.update_traces(zmax=6, zmin=-6)
fig.show()

In [147]:
fig = px.imshow(woco)
fig.update_traces(zmax=6, zmin=-6)
fig.show()